In [8]:
#imports 
# LINK: https://github.com/several27/FakeNewsCorpus/tree/master?tab=readme-ov-file
import pandas as pd
!pip install bertopic
!pip install sentence-transformers umap-learn hdbscan
%pip install bertopic

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
df_junksci_data = pd.read_csv("junk_sci_articles.csv")
df_junksci_data

,Unnamed: 0,id,domain,type,url,content,scraped_at,inserted_at,updated_at,title,authors,keywords,meta_keywords,meta_description,tags,summary,source
0,351,414.0,collective-evolution.com,junksci,http://www.collective-evolution.com/2018/01/23...,"Decades ago, when an individual claimed to wor...",2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,The Man Who Blew The Lid off of AREA 51 Shows ...,NaN,NaN,[''],Bob Lazar is the man who blew the lid open on ...,NaN,NaN,NaN
1,354,417.0,collective-evolution.com,junksci,http://www.collective-evolution.com/2018/01/24...,"Hey, you’re here, you made it! Congratulations...",2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Starbucks & McDonald’s Take Huge Steps Towards...,NaN,NaN,[''],Do these companies really care about the envir...,NaN,NaN,NaN
2,355,418.0,collective-evolution.com,junksci,http://www.collective-evolution.com/2018/01/24...,Building your network is key to a meaningful a...,2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Networking Is Now As Easy As Swiping Right,NaN,NaN,[''],Building your network is key to a meaningful a...,NaN,NaN,NaN
3,618,701.0,collective-evolution.com,junksci,http://www.collective-evolution.com/2018/01/20...,Disease has been on the rise for multiple deca...,2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,"Autophagy, Fasting & Exercise: Scientist Revea...",NaN,NaN,[''],Disease has been on the rise for multiple deca...,NaN,NaN,NaN
4,619,702.0,collective-evolution.com,junksci,http://www.collective-evolution.com/2018/01/19...,Chickens on sale for human consumption in Brit...,2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Chickens From British Supermarket Show Record ...,NaN,NaN,[''],Chickens on sale for human consumption in Brit...,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17403,9371,1007010.0,wakingtimes.com,junksci,http://www.wakingtimes.com/2012/12/05/managing...,"Stanley Popovich, Contributing Writer\n\nWakin...",2018-01-25 20:13:50.426130,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Managing Fear and Anxiety of the Unknown,NaN,NaN,[''],The prospect of not knowing if something good ...,"anxiety, mental health, pharmaceuticals, featu...",NaN,NaN
17404,5744,1023730.0,naturalnews.com,junksci,https://www.naturalnews.com/2018-01-27-make-yo...,(Natural News) Don’t get preppers wrong. They’...,2018-01-25 20:13:50.426130,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Make your own calendula extract: Step by step ...,"Zoey Sky, Saturday, January",NaN,"['environment', 'flowers', 'health', 'herbal m...",NaN,"goodhealth, natural remedies, calendula extrac...",NaN,NaN
17405,5749,1023736.0,naturalnews.com,junksci,https://www.naturalnews.com/z011701_cancer_ind...,Beating cancer -- how to take charge of your c...,2018-01-25 20:13:50.426130,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Beating cancer -- how to take charge of your c...,NaN,NaN,[''],NaN,NaN,NaN,NaN
17406,5762,1023749.0,naturalnews.com,junksci,https://www.naturalnews.com/2018-01-27-you-can...,"(Natural News) In 20 years, humans will have r...",2018-01-25 20:13:50.426130,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,"You CAN clone yourself – in about 20 years, ac...","Frances Bloomfield, Saturday, January",NaN,"['corporations', 'future science', 'robotics a...",NaN,"artificial intelligence, digital self, Cortana...",NaN,NaN


In [3]:
len(df_junksci_data)

17408

In [4]:
junk_sci_content = df_junksci_data[["content"]]
junk_sci_content

,content
0,"Decades ago, when an individual claimed to wor..."
1,"Hey, you’re here, you made it! Congratulations..."
2,Building your network is key to a meaningful a...
3,Disease has been on the rise for multiple deca...
4,Chickens on sale for human consumption in Brit...
...,...
17403,"Stanley Popovich, Contributing Writer\n\nWakin..."
17404,(Natural News) Don’t get preppers wrong. They’...
17405,Beating cancer -- how to take charge of your c...
17406,"(Natural News) In 20 years, humans will have r..."


In [6]:
#Converting the single column dataframe to a list of strings for BERTopic
#making sure that I have a list, one string/article, with any missing values removed

documents = junk_sci_content["content"].dropna().tolist()
len(documents)

17408

In [47]:
from sklearn.feature_extraction.text import CountVectorizer
from bertopic import BERTopic

vectorizer_model = CountVectorizer(stop_words="english")
topic_model = BERTopic(vectorizer_model=vectorizer_model)
result = topic_model.fit_transform(documents[:17408])

In [48]:
topics = result[0]
probs = result[1]

In [49]:
topic_info = topic_model.get_topic_info()
print(topic_info)

     Topic  Count                                       Name  \
0       -1   4776                   -1_people_time_like_just   
1        0   1903  0_looking_goodgophercom_powered_searching   
2        1   1253     1_reference_free_online_phytonutrients   
3        2    145                     2_gmo_gmos_gm_labeling   
4        3    141             3_bitcoin_banks_currency_money   
..     ...    ...                                        ...   
304    303     11                 303_col_williams_iss_space   
305    304     11             304_avocado_avocados_skin_seed   
306    305     10        305_salamanders_shrimp_snail_fossil   
307    306     10      306_happiness_happy_happier_serotonin   
308    307     10      307_dhea_adrenal_testosterone_hormone   

                                        Representation  \
0    [people, time, like, just, life, world, way, w...   
1    [looking, goodgophercom, powered, searching, i...   
2    [reference, free, online, phytonutrients, list...   

In [50]:
print(topic_model.get_topic(0))

[('looking', np.float64(0.23688713972111833)), ('goodgophercom', np.float64(0.15254716133663498)), ('powered', np.float64(0.1494640723473506)), ('searching', np.float64(0.14769892004336294)), ('interested', np.float64(0.1400773698437379)), ('search', np.float64(0.13395764740865926)), ('page', np.float64(0.1269600888528338)), ('areas', np.float64(0.12224214842133416)), ('try', np.float64(0.11572622926431052)), ('longer', np.float64(0.11443755661425072))]
